# Entrenamiento del Modelo de Detección de Ciclistas

Este notebook cubre el proceso completo para entrenar un modelo de CNN para detectar bicicletas en fotogramas de video. El proceso incluye:

1.  **Extracción de Fotogramas:** Extraer fotogramas de videos de muestra.
2.  **Generación de Archivo de Etiquetas:** Crear un `labels.csv` para que el usuario lo complete.
3.  **Carga y Preprocesamiento de Datos:** Cargar los fotogramas y etiquetas, y prepararlos para el entrenamiento.
4.  **Construcción y Entrenamiento del Modelo:** Definir y entrenar un modelo de CNN.
5.  **Guardado del Modelo:** Guardar el modelo entrenado para su uso en la aplicación.

## 1. Importar Librerías

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import shutil

## 2. Preparación de Datos

### 2.1. Extraer Fotogramas de Videos

Esta función procesa todos los videos en `../data/` y extrae un fotograma por segundo, guardándolos en `../data/frames/`.

In [ ]:
def extract_frames(video_dir='../data', output_dir='../data/frames'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    else:
        # Limpiar directorio para evitar fotogramas antiguos
        shutil.rmtree(output_dir)
        os.makedirs(output_dir)
    
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]
    
    if not video_files:
        print("No se encontraron videos en el directorio 'data/'. Por favor, añade videos para procesar.")
        return

    frame_count = 0
    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        if fps == 0:
            print(f"No se pudo leer el video {video_file}, saltando.")
            continue
            
        frame_interval = int(fps) # Extraer un fotograma por segundo
        
        while cap.isOpened():
            frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            ret, frame = cap.read()
            if not ret:
                break
            
            if frame_id % frame_interval == 0:
                frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
                cv2.imwrite(frame_filename, frame)
                frame_count += 1
        
        cap.release()
    
    print(f"Se extrajeron {frame_count} fotogramas y se guardaron en '{output_dir}'.")

# Ejecutar la extracción
extract_frames()

### 2.2. Generar Archivo de Etiquetas (para el usuario)

Este paso crea un archivo `labels.csv`. **Acción requerida por el usuario:** Debes abrir este archivo y rellenar la columna `has_bicycle` con `1` si el fotograma contiene una bicicleta y `0` si no.

In [ ]:
def generate_labels_file(frames_dir='../data/frames', output_csv='../data/labels.csv'):
    frame_files = [f for f in os.listdir(frames_dir) if f.endswith('.jpg')]
    if not frame_files:
        print("No se encontraron fotogramas. Ejecuta la extracción primero.")
        return

    df = pd.DataFrame({'frame': sorted(frame_files)})
    # El usuario debe rellenar esta columna. Por defecto, ponemos 0.
    df['has_bicycle'] = 0 
    df.to_csv(output_csv, index=False)
    print(f"Archivo de etiquetas generado en '{output_csv}'. Por favor, edítalo para etiquetar tus datos.")

# Generar el archivo de etiquetas
generate_labels_file()

**Pausa Importante:** Antes de continuar, asegúrate de haber etiquetado tus imágenes en `../data/labels.csv`.

Para fines de demostración, el siguiente código simulará que el archivo ha sido etiquetado con datos aleatorios. **En un caso de uso real, no ejecutes la siguiente celda y asegúrate de que tu archivo `labels.csv` está completo.**

In [ ]:
# --- Celda de Simulación (SOLO PARA DEMO) ---
labels_path = '../data/labels.csv'
if os.path.exists(labels_path):
    df_labels = pd.read_csv(labels_path)
    if not df_labels.empty:
        # Simular etiquetado aleatorio
        df_labels['has_bicycle'] = np.random.randint(0, 2, df_labels.shape[0])
        df_labels.to_csv(labels_path, index=False)
        print("Simulación: Se han añadido etiquetas aleatorias a 'labels.csv'.")

## 3. Cargar y Preprocesar Datos

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

labels_df = pd.read_csv('../data/labels.csv')
labels_df['has_bicycle'] = labels_df['has_bicycle'].astype(str)

train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['has_bicycle'])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../data/frames',
    x_col='frame',
    y_col='has_bicycle',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='../data/frames',
    x_col='frame',
    y_col='has_bicycle',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

## 4. Construcción y Entrenamiento del Modelo

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Entrenar el modelo (si hay datos)
if train_generator.n > 0 and validation_generator.n > 0:
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.n // BATCH_SIZE,
        epochs=10, # En un caso real, usar más épocas
        validation_data=validation_generator,
        validation_steps=validation_generator.n // BATCH_SIZE
    )
else:
    print("No hay suficientes datos para entrenar. Por favor, verifica tus datos y etiquetas.")

## 5. Guardar el Modelo Entrenado

In [ ]:
model.save('../bicycle_detection_model.h5')
print("Modelo guardado como '../bicycle_detection_model.h5'")